<a href="https://colab.research.google.com/github/alphabeen/codingnote/blob/main/cats_and_dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Load

In [2]:
import tensorflow_datasets as tfds
import tensorflow as tf

dataset_name = 'cats_vs_dogs'
train_dataset=tfds.load(name=dataset_name,split='train[:80%]',shuffle_files=True)
valid_dataset=tfds.load(name=dataset_name,split='train[80%:]',shuffle_files=True)

In [3]:
train_dataset

<PrefetchDataset element_spec={'image': TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), 'image/filename': TensorSpec(shape=(), dtype=tf.string, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>

#Preprocessing

In [4]:
def preprocess(data):
    x = data['image']
    y = data['label']
    x = x / 255
    x = tf.image.resize(x, size=(224, 224))

    return x, y

#Modeling

In [5]:
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

train_data = train_dataset.map(preprocess).batch(32)
valid_data = valid_dataset.map(preprocess).batch(32)
model = Sequential([
        Conv2D(32, (3, 3), input_shape=(224, 224, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(32, (3, 3), activation='relu'),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),        
        Conv2D(32, (3, 3), activation='relu'),        
        MaxPooling2D(2, 2),
        Flatten(),
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(2, activation='softmax'),
])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 32)        18464     
                                                                 
 conv2d_3 (Conv2D)           (None, 50, 50, 32)        9

#Training

In [7]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
checkpoint_path = "my_checkpoint.ckpt"
checkpoint = ModelCheckpoint(filepath=checkpoint_path, 
                            save_weights_only=True, 
                            save_best_only=True, 
                            monitor='val_loss', 
                            verbose=1)
model.fit(train_data,
          validation_data=(valid_data),
          epochs=3,
          callbacks=[checkpoint],
          )
model.load_weights(checkpoint_path)

Epoch 1/3
582/582 [==============================] - ETA: 0s - loss: 0.6663 - acc: 0.5870
Epoch 1: val_loss improved from inf to 0.58827, saving model to my_checkpoint.ckpt
582/582 [==============================] - 52s 83ms/step - loss: 0.6663 - acc: 0.5870 - val_loss: 0.5883 - val_acc: 0.6920
Epoch 2/3
582/582 [==============================] - ETA: 0s - loss: 0.5708 - acc: 0.7012
Epoch 2: val_loss improved from 0.58827 to 0.50141, saving model to my_checkpoint.ckpt
582/582 [==============================] - 45s 76ms/step - loss: 0.5708 - acc: 0.7012 - val_loss: 0.5014 - val_acc: 0.7571
Epoch 3/3
581/582 [============================>.] - ETA: 0s - loss: 0.4944 - acc: 0.7617
Epoch 3: val_loss improved from 0.50141 to 0.47725, saving model to my_checkpoint.ckpt
582/582 [==============================] - 47s 81ms/step - loss: 0.4943 - acc: 0.7618 - val_loss: 0.4772 - val_acc: 0.7736


#Evaluate

In [8]:
loss, acc = model.evaluate(valid_data, verbose=2)
print("test loss: %f, test accuracy: %f" %(loss, acc*100))

146/146 - 7s - loss: 0.4772 - acc: 0.7736 - 7s/epoch - 50ms/step
test loss: 0.477247, test accuracy: 77.364576


#Pretrained_Model

In [16]:
from tensorflow.keras.applications import VGG16


transfer_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
transfer_model.trainable=False


VGG_model = Sequential([
        transfer_model,
        Flatten(),
        Dropout(0.5),
        Dense(512,activation='relu'),
        Dense(128,activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
])

In [13]:
VGG_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_2 (Flatten)         (None, 25088)             0         
                                                                 
 dropout_2 (Dropout)         (None, 25088)             0         
                                                                 
 dense_6 (Dense)             (None, 512)               12845568  
                                                                 
 dense_7 (Dense)             (None, 128)               65664     
                                                                 
 dense_8 (Dense)             (None, 2)                 258       
                                                                 
Total params: 27,626,178
Trainable params: 12,911,490


In [14]:
VGG_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
checkpoint_path_VGG = "my_checkpoint_VGG.ckpt"
checkpoint_VGG = ModelCheckpoint(filepath=checkpoint_path_VGG, 
                            save_weights_only=True, 
                            save_best_only=True, 
                            monitor='val_loss', 
                            verbose=1)
VGG_model.fit(train_data,
          validation_data=(valid_data),
          epochs=3,
          callbacks=[checkpoint_VGG],
          )
VGG_model.load_weights(checkpoint_path_VGG)

Epoch 1/3
582/582 [==============================] - ETA: 0s - loss: 0.2793 - acc: 0.8867
Epoch 1: val_loss improved from inf to 0.16973, saving model to my_checkpoint_VGG.ckpt
582/582 [==============================] - 124s 212ms/step - loss: 0.2793 - acc: 0.8867 - val_loss: 0.1697 - val_acc: 0.9308
Epoch 2/3
582/582 [==============================] - ETA: 0s - loss: 0.1771 - acc: 0.9261
Epoch 2: val_loss improved from 0.16973 to 0.16706, saving model to my_checkpoint_VGG.ckpt
582/582 [==============================] - 120s 205ms/step - loss: 0.1771 - acc: 0.9261 - val_loss: 0.1671 - val_acc: 0.9278
Epoch 3/3
582/582 [==============================] - ETA: 0s - loss: 0.1562 - acc: 0.9359
Epoch 3: val_loss improved from 0.16706 to 0.15818, saving model to my_checkpoint_VGG.ckpt
582/582 [==============================] - 119s 204ms/step - loss: 0.1562 - acc: 0.9359 - val_loss: 0.1582 - val_acc: 0.9321


In [15]:
loss, acc = VGG_model.evaluate(valid_data, verbose=2)
print("test loss: %f, test accuracy: %f" %(loss, acc*100))

146/146 - 24s - loss: 0.1582 - acc: 0.9321 - 24s/epoch - 161ms/step
test loss: 0.158182, test accuracy: 93.207222
